In [66]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import serviceHandlers as handlers
from serviceHandlers import NamSorEndpoint


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
# Getting a dataset

df = pd.read_csv("../../data/all.csv")
df = df[df['hasMiddleName']==False].iloc[0:10]
df

,Unnamed: 0,fullName,firstName,lastName,gender,isoCountry,continent,birthYear,hasMiddleName,hasNoLastName,source
0,0,Clara Benson,Clara,Benson,female,GH,Africa,2000,False,False,wikidata
2,2,Dalia Ziada,Dalia,Ziada,female,EG,Africa,1982,False,False,wikidata
3,3,Fatou Haidara,Fatou,Haidara,female,ML,Africa,1962,False,False,wikidata
4,4,Claude Haffner,Claude,Haffner,female,NaN,NaN,1976,False,False,wikidata
7,7,Diallo Sène,Diallo,Sène,female,ML,Africa,1952,False,False,wikidata
8,8,Fatoumata Sylla,Fatoumata,Sylla,female,ML,Africa,1954,False,False,wikidata
11,11,Manuella Ollo,Manuella,Ollo,female,NaN,NaN,1985,False,False,wikidata
12,12,Awa Traoré,Awa,Traoré,female,ML,Africa,1980,False,False,wikidata
17,17,Lorna Doorman,Lorna,Doorman,female,ZW,Africa,1998,False,False,wikidata
18,18,Enayat al-Zayyat,Enayat,al-Zayyat,female,EG,Africa,1936,False,False,wikidata


# Testing GenderizeIoHandler

In [68]:
# genderizeIO_handler = handlers.GenderizeIoHandler(datasource=df, hasSubscription=False)
# genderizeIO_handler.get_prediction(True)

# Testing GenderAPI

In [69]:
# genderAPI_IO_Handler = handlers.GenderAPI_IO_Handler(datasource=df)

# genderAPI_IO_Handler.callAPI(useLocalization=True)

# Testing GenderGuesser

In [70]:
# genderGuesserHandler = handlers.GenderGuesserHandler(df)

# genderGuesserHandler.get_prediction(False)

# Testing GenderAPI.com 

In [71]:
# genderAPI_com_handler = handlers.GenderAPI_com_Handler(df)

# genderAPI_com_handler.get_prediction(True, True)

# Testing NamSor Handler

In [72]:
# namSor_handler = handlers.NamSorHandler(df)

# namSor_handler.get_prediction(endpoint=NamSorEndpoint.FULL_NAME)

# Testing NameAPI

In [73]:
# nameAPI_handler = handlers.NameAPIHandler(df)

# await nameAPI_handler.get_prediction(True)

In [74]:
import serviceWrapperAsync as wrapper
from serviceWrapperAsync import NamSorEndpoint as endpoint

In [75]:
handler = wrapper.GenderizeWrapper(df)
await handler.get_prediction_async(useLocalization=True)

,index,fullName,namePassed,correctGender,predictedGender,localization,useLocalization,serviceUsed,extraCount,extraProbability
0,0,Clara Benson,Clara,female,female,GH,True,genderize.IO,247083,0.99
1,1,Dalia Ziada,Dalia,female,female,EG,True,genderize.IO,87214,0.99
2,2,Fatou Haidara,Fatou,female,female,ML,True,genderize.IO,25477,0.98
3,3,Claude Haffner,Claude,female,male,NaN,True,genderize.IO,161269,0.93
4,4,Diallo Sène,Diallo,female,male,ML,True,genderize.IO,8032,0.82
5,5,Fatoumata Sylla,Fatoumata,female,female,ML,True,genderize.IO,15498,0.99
6,6,Manuella Ollo,Manuella,female,female,NaN,True,genderize.IO,12717,0.99
7,7,Awa Traoré,Awa,female,female,ML,True,genderize.IO,16472,0.93
8,8,Lorna Doorman,Lorna,female,female,ZW,True,genderize.IO,53554,1.00
9,9,Enayat al-Zayyat,Enayat,female,male,EG,True,genderize.IO,1276,0.56
